# 1. Реализация алгоритма синдромного вложения на основе кодов Хэмминга

## 1.1 Алгоритм вложения:
Реализуйте функцию вложения, которая принимает исходный
контейнер x (блок из n битов) и встраиваемое скрытое сообщение m (требуемый
синдром). Функция должна находить изменённый контейнер x̃ таким образом, чтобы
выполнялось условие:
Hx̃^T = m,
где H — проверочная матрица кода Хэмминга. При этом количество изменений должно быть минимальным (d(x, ̃x̃) ≤ 1).
Используйте метод, основанный на нахождении вектора ошибки e веса ≤ 1, для
которого выполнялось бы условие
He^T = m − Hx^T
В этом случае x̃= x + e


In [1]:
def str_to_matrix(s):
    result = matrix(1, len(s))
    for i in range(len(s)):
        result[0, i] = int(s[i])
    return result
def matrix_to_str(m_matrix):
    return  ''.join(str(bit) for bit in m_matrix[0])

In [2]:
def H_product(x):
    s = 0
    for i in range(1, x.ncols() + 1):  
        if x[0, i-1] == 1: 
            s = i ^^ s
    return s

def hidden_message(x, m):
    str_m = matrix_to_str(m)
    pos = H_product(x) ^^ int(str_m, 2)
    if pos != 0:      
        x[0, pos-1] = x[0, pos-1] ^^ 1 
    return x

x_matrix = str_to_matrix('0010010')
m_matrix = str_to_matrix('101')

hidden_message(x_matrix, m_matrix)

[0 0 1 0 0 1 0]

## 1.2 Алгоритм извлечения:
Реализуйте функцию извлечения, которая вычисляет скрытое
сообщение m из стегоконтейнера x̃путём прямого вычисления синдрома:
m = Hx̃
T

In [3]:
def recovery_hidden_message(x, message_length):
    return str_to_matrix(bin(H_product(hidden_message(x_matrix, m_matrix)))[2:].zfill(message_length))

In [4]:
recovery_hidden_message(hidden_message(x_matrix, m_matrix), 3)

[1 0 1]

# 2. Интеграция с медиа-контейнером (LSB)

Реализуйте комбинацию описанного выше алгоритма синдромного вложения с методом
LSB (Least Significant Bit) для работы с выбранным медиа-контейнером (изображение
(PNG), аудио (WAV) или видео).

In [5]:
def hidden_message_in_image(container, message, r=3):
    n = (2**r) - 1
   
    result = []

    if message.ncols()%r!=0:
        zeros_to_add = matrix([[0] * (r - (message.ncols() % r)) for _ in range(message.nrows())])
        message = matrix(np.hstack([zeros_to_add, message]))
    ind = 0
    for i, j in zip(range(0, container.ncols(), n), range(0, message.ncols(), r)):
        hidden_result = hidden_message(container[0, i:i+n], message[0, j:j+r])
        result.extend([hidden_result[0, k] for k in range(hidden_result.ncols())])
        ind = i + n
    
    if ind < container.ncols():
        result.extend([container[0, k] for k in range(ind, container.ncols())])
    
    return matrix(result)  

In [6]:
def extract_message_from_image(stego_image_path, message_length, r=3):
    stego_img = Image.open(stego_image_path).convert('L')
    stego_matrix = matrix(np.array(stego_img))
    stego_matrix = stego_matrix.apply_map(lambda x: x & 1)
    
    container = matrix(list(stego_matrix.list()))
    
    n = (2**r) - 1 
    k = (2**r) - r - 1 
    
    extracted_bits = []
    
    num_blocks = (message_length+(r - message_length%r)%r) // r 
    for i in range(0, num_blocks * n, n):
        if i + n > container.ncols():
            break
            
        block = container[0, i:i+n]
        
        syndrome = H_product(block)
        
        syndrome_bits = bin(syndrome)[2:].zfill(r)
        
        extracted_bits.extend([int(bit) for bit in syndrome_bits])
    
    extracted_bits = extracted_bits[len(extracted_bits) - message_length:]
    
    return matrix([extracted_bits])

In [7]:
from PIL import Image
import numpy as np

def image_to_LSB_container(filename):
    img = Image.open(filename).convert('L')  
    M = matrix(np.array(img))
    M = M.apply_map(lambda x: x & 1)
    return matrix(list(M.list()))
    
img = Image.open('image.png').convert('L')

imag = matrix(np.array(img))
M = image_to_LSB_container('image.png')

# для тестов NIST
filepath = os.path.join('NIST-Statistical-Test-Suite', 'sts', 'data', 'input_bits.txt')
with open(filepath, 'w') as f:
    f.write(''.join(str(bit) for bit in M[0]))

In [8]:
#message = str_to_matrix('10')
message = str_to_matrix('10010010100100110101001011111111111111111111110010101000100001111')
#message = str_to_matrix('100100101') 
mm = hidden_message_in_image(M, message, 4)

filepath = os.path.join('NIST-Statistical-Test-Suite', 'sts', 'data', 'new_bits.txt')
with open(filepath, 'w') as f:
    f.write(''.join(str(bit) for bit in mm[0]))

In [9]:
k = 0
for i in range(imag.nrows()):
    for j in range(imag.ncols()):
        imag[i,j] = (imag[i,j] & 0xFE) | mm[0, k]
        k+=1
image_array = [[int(imag[i, j]) for j in range(imag.ncols())]  
               for i in range(imag.nrows())]

image_np = np.array(image_array, dtype=np.uint8)

i = Image.fromarray(image_np)
i.save('output.png')


In [10]:
recovery_message = extract_message_from_image('output.png',message.ncols(), 4)
recovery_message == message


True

# 3. Стеганоанализ
Проведите статистический анализ распределения младших битов стегоконтейнера на
предмет аномалий. Можно, например, воспользоваться NIST Statistical Test Suite.


In [11]:
%cd NIST-Statistical-Test-Suite/sts

/mnt/c/Users/Амалия/NIST-Statistical-Test-Suite/sts


In [13]:
!./assess 68213 <../test/input_bits_test.txt

           G E N E R A T O R    S E L E C T I O N 
           ______________________________________

    [0] Input File                 [1] Linear Congruential
    [2] Quadratic Congruential I   [3] Quadratic Congruential II
    [4] Cubic Congruential         [5] XOR
    [6] Modular Exponentiation     [7] Blum-Blum-Shub
    [8] Micali-Schnorr             [9] G Using SHA-1

   Enter Choice: 

		User Prescribed Input File: 
                S T A T I S T I C A L   T E S T S
                _________________________________

    [01] Frequency                       [02] Block Frequency
    [03] Cumulative Sums                 [04] Runs
    [05] Longest Run of Ones             [06] Rank
    [07] Discrete Fourier Transform      [08] Nonperiodic Template Matchings
    [09] Overlapping Template Matchings  [10] Universal Statistical
    [11] Approximate Entropy             [12] Random Excursions
    [13] Random Excursions Variant       [14] Serial
    [15] Linear Complexity

         INSTRUCTI

In [17]:
!cat experiments/AlgorithmTesting/finalAnalysisReport.txt

------------------------------------------------------------------------------
RESULTS FOR THE UNIFORMITY OF P-VALUES AND THE PROPORTION OF PASSING SEQUENCES
------------------------------------------------------------------------------
   generator is <data/input_bits.txt>
------------------------------------------------------------------------------
 C1  C2  C3  C4  C5  C6  C7  C8  C9 C10  P-VALUE  PROPORTION  STATISTICAL TEST
------------------------------------------------------------------------------
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  Frequency
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  BlockFrequency
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  CumulativeSums
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  CumulativeSums
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  Runs
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  LongestRun
 11   0   0   0   0   0   0

In [18]:
!./assess 68213 <../test/new_bits_test.txt

           G E N E R A T O R    S E L E C T I O N 
           ______________________________________

    [0] Input File                 [1] Linear Congruential
    [2] Quadratic Congruential I   [3] Quadratic Congruential II
    [4] Cubic Congruential         [5] XOR
    [6] Modular Exponentiation     [7] Blum-Blum-Shub
    [8] Micali-Schnorr             [9] G Using SHA-1

   Enter Choice: 

		User Prescribed Input File: 
                S T A T I S T I C A L   T E S T S
                _________________________________

    [01] Frequency                       [02] Block Frequency
    [03] Cumulative Sums                 [04] Runs
    [05] Longest Run of Ones             [06] Rank
    [07] Discrete Fourier Transform      [08] Nonperiodic Template Matchings
    [09] Overlapping Template Matchings  [10] Universal Statistical
    [11] Approximate Entropy             [12] Random Excursions
    [13] Random Excursions Variant       [14] Serial
    [15] Linear Complexity

         INSTRUCTI

In [19]:
!cat experiments/AlgorithmTesting/finalAnalysisReport.txt

------------------------------------------------------------------------------
RESULTS FOR THE UNIFORMITY OF P-VALUES AND THE PROPORTION OF PASSING SEQUENCES
------------------------------------------------------------------------------
   generator is <data/new_bits.txt>
------------------------------------------------------------------------------
 C1  C2  C3  C4  C5  C6  C7  C8  C9 C10  P-VALUE  PROPORTION  STATISTICAL TEST
------------------------------------------------------------------------------
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  Frequency
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  BlockFrequency
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  CumulativeSums
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  CumulativeSums
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  Runs
 11   0   0   0   0   0   0   0   0   0  0.000000 *    0/11   *  LongestRun
 11   0   0   0   0   0   0  

# 4. Секретный ключ и псевдослучайный выбор
Реализуйте механизм, повышающий безопасность вложения и затрудняющий стеганоанализ, путём псевдослучайного выбора областей вложения, управляемого некоторым
секретным ключом.
## Псевдослучайный выбор областей:
- Генерация зерна (Seed): Разработайте процедуру, которая некоторый секрет K
(пароль пользователя) для генерации надёжного начального значения (зерна) для
генератора псевдослучайных чисел (ГПСЧ). Рекомендуется использовать криптографическую хеш-функцию (например, SHA-256) для получения детерминированного и надёжного зерна из ключа K.
Seed = Hash(K)

- Генерация индексов: Используйте Seed для инициализации криптографически безопасного ГПСЧ (CSPRNG). ГПСЧ должен сгенерировать псевдослучайную последовательность уникальных индексов I = {i1, i2, … , iL}, соответствующих позициям
пикселей/отсчётов в медиафайле, в LSB которых будет производиться вложение.

- Вложение по ключу: Применяйте алгоритм синдромного вложения, реализованный ранее, только к тем битам LSB, которые соответствуют индексам, сгенерированным ГПСЧ. Это обеспечивает рассредоточенность скрытых данных по всему
контейнеру, что значительно повышает устойчивость к стеганоанализу, основанному на локальных аномалиях.

- Извлечение по ключу: Процедура извлечения должна использовать тот же самый
ключ K для повторной генерации идентичной последовательности индексов I.

In [35]:
%cd ..

/mnt/c/Users/Амалия


In [36]:
import hashlib

In [1]:
sample(range(100), 10)

[94, 69, 99, 80, 36, 13, 86, 44, 12, 76]

In [37]:
def hidden_message_in_image_with_key(container, message, key, r=3):
    n = (2**r) - 1
    
    set_random_seed(int(hashlib.sha256(key.encode()).hexdigest(), 16) % (2**32))
    
    total_bits = container.ncols()
    
    if message.ncols() % r != 0:
        zeros_to_add = matrix([[0] * (r - (message.ncols() % r)) for _ in range(message.nrows())])
        message = matrix(np.hstack([zeros_to_add, message]))
    
    num_blocks = message.ncols() // r
    num_indices = num_blocks * n
    
    all_indices = list(range(total_bits))
    shuffle(all_indices)  
    indices = all_indices[:num_indices] 
    indices.sort()
    container_list = list(container[0])
    
    for block_idx in range(num_blocks):
        start_msg = block_idx * r
        end_msg = start_msg + r
        msg_block = message[0, start_msg:end_msg]
        
        start_idx = block_idx * n
        block_indices = indices[start_idx:start_idx + n]
        
        container_block = matrix([container_list[i] for i in block_indices])
        
        modified_block = hidden_message(container_block, msg_block)
        
        for i, idx in enumerate(block_indices):
            container_list[idx] = modified_block[0, i]
    
    return matrix([container_list])

In [38]:
def extract_message_from_image_with_key(stego_image_path, message_length, key, r=3):
    stego_img = Image.open(stego_image_path).convert('L')
    stego_matrix = matrix(np.array(stego_img))
    stego_matrix = stego_matrix.apply_map(lambda x: x & 1)
    
    container = matrix(list(stego_matrix.list()))
    total_bits = container.ncols()
    
    set_random_seed(int(hashlib.sha256(key.encode()).hexdigest(), 16) % (2**32))
    
    n = (2**r) - 1
    num_blocks = (message_length + (r - message_length % r) % r) // r
    num_indices = num_blocks * n
    
    all_indices = list(range(total_bits))
    shuffle(all_indices) 
    indices = all_indices[:num_indices]  
    indices.sort()
    
    extracted_bits = []
    
    for block_idx in range(num_blocks):
        start_idx = block_idx * n
        block_indices = indices[start_idx:start_idx + n]
        
        block = matrix([container[0, i] for i in block_indices])
        
        syndrome = H_product(block)
        syndrome_bits = bin(syndrome)[2:].zfill(r)
        
        extracted_bits.extend([int(bit) for bit in syndrome_bits])
    
    extracted_bits = extracted_bits[len(extracted_bits) - message_length:]
    
    return matrix([extracted_bits])

In [39]:
img = Image.open('image.png').convert('L')

imag = matrix(np.array(img))
M = image_to_LSB_container('image.png')


In [40]:
#message = str_to_matrix('10')
message = str_to_matrix('10010010100100110101001011111111111111111111110010101000100001111')
#message = str_to_matrix('100100101') 
#message = str_to_matrix('111111111111111111111111111111111111111111')

key = "Kirill Vladimirovich"
wrong_key = "Vladimir Kirillovich"

mm = hidden_message_in_image_with_key(M, message,key, 4)


In [41]:
k = 0
for i in range(imag.nrows()):
    for j in range(imag.ncols()):
        imag[i,j] = (imag[i,j] & 0xFE) | mm[0, k]
        k+=1
image_array = [[int(imag[i, j]) for j in range(imag.ncols())]  
               for i in range(imag.nrows())]

image_np = np.array(image_array, dtype=np.uint8)

i = Image.fromarray(image_np)
i.save('output2.png')


In [42]:
recovery_message = extract_message_from_image_with_key('output2.png',message.ncols(),key, 4)
recovery_message == message

True

In [43]:
recovery_message = extract_message_from_image_with_key('output2.png',message.ncols(),wrong_key, 4)
recovery_message == message

False